In [11]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [12]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(base_url="https://api.fanar.qa/v1",api_key=os.environ["FANAR_API_KEY"],)
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="azure"
deployment = APIKeyManager(model_type, "./azure.env")


In [13]:
class ImprovedPodcastOutlineGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_outline(self, topic, information, host_persona, guest_persona, style="حواري", duration="15 دقيقة"):
        
        # Style-specific guidance
        style_prompts = {
            "حواري": """
أسلوب حواري:
- ركز على الحوار الطبيعي والتفاعل الشخصي بين المقدم والضيف
- أضف لحظات تداخل وتفاعل عفوي
- استخدم لغة ودية ومألوفة
- اجعل النقاش يبدو كمحادثة بين أصدقاء
- أكثر من الأسئلة الشخصية والتجارب الذاتية""",
            
            "تعليمي": """
أسلوب تعليمي:
- اهتم بتقديم المعلومات بطريقة منظمة ومفصلة
- استخدم أمثلة توضيحية وتشبيهات مفهومة
- اجعل الضيف يشرح المفاهيم خطوة بخطوة
- أضف أسئلة استيضاحية من المقدم
- ركز على الفهم العميق للموضوع مع الحفاظ على الطبيعية""",
            
            "ترفيهي": """
أسلوب ترفيهي:
- أضف عناصر مرحة وقصص شخصية طريفة
- استخدم الفكاهة المناسبة والتعليقات الخفيفة
- اجعل النقاش حيوياً ومليئاً بالطاقة
- أضف مواقف مضحكة أو غريبة مرتبطة بالموضوع
- ركز على الجانب الإنساني والممتع من الموضوع""",
            
            "تحليلي": """
أسلوب تحليلي:
- ركز على التحليل العميق والنقاش المتخصص
- استخدم بيانات وإحصائيات ومراجع علمية
- اطرح أسئلة تحليلية معقدة
- ناقش التحديات والحلول بتفصيل
- اجعل النقاش فكرياً ومتعمقاً مع الحفاظ على الوضوح"""
        }
        
        style_guidance = style_prompts.get(style, "حافظ على التوازن بين المحتوى والتفاعل")
        
        # Duration-specific guidance
        duration_guidance = ""
        if "45" in duration or "طويلة" in duration:
            duration_guidance = "نظراً لطول الحلقة، أضف المزيد من التفاصيل والأمثلة والنقاط الفرعية في كل قسم."
        elif "15" in duration or "قصيرة" in duration:
            duration_guidance = "نظراً لقصر الحلقة، ركز على النقاط الأساسية واجعل المحتوى مكثفاً ومباشراً."
        else:
            duration_guidance = "اجعل المحتوى متوازناً ومناسباً لحلقة متوسطة الطول."

        prompt = f"""
أنت خبير في تصميم حلقات البودكاست العربية المميزة. مهمتك إنشاء مخطط تفصيلي لحلقة بودكاست حول موضوع: '{topic}'

معلومات الحلقة:
- المدة المطلوبة: {duration}
- الأسلوب المطلوب: {style}

{style_guidance}

{duration_guidance}

استخدم الشخصيات التالية بدقة:
- المقدم: {host_persona}
- الضيف: {guest_persona}

اعتمد على المعلومات التالية في المخطط:
{information}

أنشئ مخططاً بصيغة JSON يتضمن العناصر التالية لضمان حوار طبيعي وتلقائي:

{{
    "episode_topic": "الموضوع الرئيسي للحلقة",
    "personas": {{
        "host": {{
            "name": "اسم المقدم",
            "background": "خلفية المقدم",
            "speaking_style": "أسلوب التحدث والنبرة"
        }},
        "guest": {{
            "name": "اسم الضيف", 
            "background": "خلفية الضيف",
            "speaking_style": "أسلوب التحدث والنبرة"
        }}
    }},
    "conversation_flow": {{
        "intro1": {{
            "description": "مقدمة عامة للبودكاست وترحيب بالمستمعين",
            "tone": "نبرة ترحيبية حماسية",
            "script_elements": [
                "ترحيب عام بالمستمعين",
                "تعريف مختصر بالبودكاست",
                "جملة تشويقية للحلقة"
            ],
            "spontaneity_triggers": [
                "أهلاً وسهلاً بكم معنا مرة أخرى...",
                "يسعدني أن ألتقي بكم اليوم..."
            ]
        }},
        "intro2": {{
            "description": "تقديم موضوع اليوم والترحيب بالضيف",
            "tone": "نبرة تقديمية رسمية ودافئة",
            "script_elements": [
                "تقديم موضوع الحلقة بطريقة مشوقة",
                "ترحيب حار بالضيف",
                "تعريف مختصر بخلفية الضيف"
            ],
            "cultural_hooks": [
                "روابط ثقافية للموضوع",
                "مراجع محلية أو إقليمية"
            ],
            "transition_phrases": [
                "موضوعنا اليوم يتناول...",
                "ويسعدني أن أستضيف معنا...",
                "دعونا نرحب بضيفنا الكريم..."
            ]
        }},
        "main_discussion": [
            {{
                "point_title": "عنوان النقطة الأولى",
                "personal_angle": "كيف ترتبط هذه النقطة بخلفية الشخصيات",
                "spontaneous_triggers": [
                    "محفزات تلقائية مرتبطة بهذه النقطة تحديداً",
                    "عبارات طبيعية للحديث عن هذا الجانب من الموضوع"
                ],
                "disagreement_points": "نقاط اختلاف أو جدل محتملة مرتبطة بهذه النقطة تحديداً",
                "cultural_references": [
                    "أمثال ومقولات مرتبطة بهذا الجانب من الموضوع",
                    "مراجع ثقافية محددة ذات صلة مباشرة"
                ],
                "natural_transitions": "عبارات انتقال طبيعية للنقطة التالية مرتبطة بالموضوع",
                "emotional_triggers": "محفزات عاطفية مرتبطة بهذا الجانب من الموضوع"
            }},
            {{
                "point_title": "عنوان النقطة الثانية",
                "personal_angle": "الربط الشخصي بالموضوع",
                "spontaneous_triggers": [
                    "في الواقع أريد أن أضيف شيئاً...",
                    "لا بد أن أشارككم رأيي في هذا..."
                ],
                "disagreement_points": "نقاط جدل صحية",
                "cultural_references": [
                    "مراجع ثقافية ذات صلة",
                    "تجارب مشتركة"
                ],
                "natural_transitions": "انتقالات سلسة",
                "emotional_triggers": "محفزات عاطفية"
            }},
            {{
                "point_title": "عنوان النقطة الثالثة",
                "personal_angle": "زاوية شخصية للموضوع",
                "spontaneous_triggers": [
                    "انتظروا، دعوني أوضح نقطة مهمة...",
                    "هذا الموضوع يهمني شخصياً لأن..."
                ],
                "disagreement_points": "نقاط اختلاف بناءة",
                "cultural_references": [
                    "حكم ومأثورات",
                    "قصص تراثية"
                ],
                "natural_transitions": "عبارات ربط طبيعية",
                "emotional_triggers": "لحظات تأثر طبيعية"
            }}
        ],
        "spontaneous_moments": {{
            "natural_interruptions": [
                "نقاط تداخل طبيعية في المحادثة",
                "لحظات مقاطعة إيجابية"
            ],
            "emotional_reactions": [
                "محفزات للتعبير العاطفي",
                "لحظات تأثر حقيقية"
            ],
            "personal_stories": [
                "مواضع لحكايات شخصية",
                "تجارب ذاتية مرتبطة بالموضوع"
            ],
            "humorous_moments": [
                "لحظات خفة دم طبيعية",
                "تعليقات مرحة مناسبة"
            ]
        }},
        "personality_interactions": {{
            "host_strengths": "نقاط قوة المقدم في الحوار",
            "guest_expertise": "مجالات خبرة الضيف",
            "natural_chemistry": "كيف تتفاعل الشخصيتان طبيعياً",
            "tension_points": "نقاط توتر صحية للحوار",
            "collaboration_moments": "لحظات تعاون وتفاهم"
        }},
        "closing": {{
            "conclusion": {{
                "main_takeaways": "النقاط الرئيسية المحددة التي يجب تلخيصها",
                "guest_final_message": "الرسالة الختامية المحددة للضيف",
                "host_closing_thoughts": "أفكار المقدم الختامية المحددة",
                "summary_tone": "نبرة التلخيص المطلوبة"
            }},
            "outro": {{
                "guest_appreciation": "النص الفعلي لشكر الضيف",
                "audience_thanks": "النص الفعلي لشكر المستمعين",
                "call_to_action": "الدعوة المحددة للتفاعل",
                "next_episode_teaser": "التشويق المحدد للحلقة القادمة",
                "final_goodbye": "النص الفعلي للوداع",
                "engagement_question": "السؤال المحدد للجمهور"
            }}
        }}
    }},
    "cultural_context": {{
        "proverbs_sayings": [
            "أمثال وحكم مرتبطة مباشرة بالموضوع المطروح",
            "مقولات مأثورة ذات صلة بالموضوع المحدد"
        ],
        "regional_references": [
            "مراجع محلية وإقليمية متعلقة بالموضوع",
            "تجارب عربية أو محلية مرتبطة بالموضوع"
        ],
        "shared_experiences": [
            "تجارب جماعية عربية مرتبطة بالموضوع",
            "ذكريات أو مواقف مشتركة متعلقة بالموضوع"
        ],
        "contemporary_relevance": [
            "أحداث جارية مرتبطة بالموضوع",
            "تطورات حديثة في المجال المطروح"
        ]
    }},
    "language_style": {{
        "formality_level": "مستوى الرسمية في اللغة",
        "dialect_touches": "لمسات لهجية خفيفة مناسبة",
        "vocabulary_richness": "ثراء المفردات المستخدمة",
        "sentence_variety": "تنويع في طول وتركيب الجمل"
    }},
    "technical_notes": {{
        "pacing_guidance": "إرشادات حول إيقاع الحديث",
        "pause_points": "نقاط توقف طبيعية",
        "emphasis_moments": "لحظات تأكيد وتشديد",
        "background_music_cues": "إشارات للموسيقى التصويرية"
    }}
}}

تأكد من أن المخطط يشجع على:
1. الحوار الطبيعي والتلقائي
2. إدماج شخصيات الأفراد بشكل أصيل
3. استخدام اللغة العربية الفصحى بطريقة طبيعية
4. إضافة لمسات ثقافية مناسبة
5. خلق لحظات عاطفية حقيقية

مهم جداً - متطلبات المحتوى:
- لا تكتب أوصافاً عامة مثل "ترحيب عام" أو "تقديم الموضوع"
- اكتب المحتوى الفعلي والنصوص المحددة التي سيقولها المقدم والضيف
- كل قيمة يجب أن تحتوي على نص حقيقي وليس وصفاً لما يجب فعله
- استخدم أسماء الشخصيات الفعلية (أحمد، نور) في النصوص
- اجعل كل محتوى مرتبطاً مباشرة بموضوع السيارات ذاتية القيادة
- أرجع المخطط بصيغة JSON صحيحة فقط
- لا تضع علامات ```json في البداية أو ``` في النهاية
- لا تكتب أي نص إضافي قبل أو بعد JSON

مثال على ما هو مطلوب:
بدلاً من: "ترحيب عام بالمستمعين"
اكتب: "أهلاً وسهلاً بكم في حلقة جديدة من بودكاست التقنية والمستقبل، أنا أحمد المصري معكم اليوم"

بدلاً من: "تقديم الموضوع"
اكتب: "اليوم سنتحدث عن السيارات التي تقود نفسها، هذه التقنية المذهلة التي قد تغير حياتنا كلياً"
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تصميم البودكاست العربية وتخصص في خلق محتوى طبيعي وتلقائي باللغة العربية الفصحى. الأسلوب المطلوب: {style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        
        return response.choices[0].message.content

    def generate_outline_with_style(self, topic, information, host_persona, guest_persona, style="حواري", duration="30 دقيقة"):
        """
        Generate outline with specific style considerations
        Styles: حواري (conversational), تعليمي (educational), ترفيهي (entertaining), تحليلي (analytical)
        """
        # Now this method just calls the main method which already handles style
        return self.generate_outline(topic, information, host_persona, guest_persona, style, duration)

    def validate_outline(self, outline_json):
        """
        Validate that the generated outline contains all required elements
        """
        required_keys = [
            "episode_topic", "personas", "conversation_flow", 
            "cultural_context", "language_style", "technical_notes"
        ]
        
        try:
            import json
            outline = json.loads(outline_json)
            
            missing_keys = []
            for key in required_keys:
                if key not in outline:
                    missing_keys.append(key)
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Outline validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

In [14]:
# ---------------------------------------------------------------
#  New sample run: Quantum Computing episode (entertaining style)
# ---------------------------------------------------------------

outline_generator = ImprovedPodcastOutlineGenerator(deployment, "fanar")

topic = "الحوسبة الكمومية: ثورة التكنولوجيا القادمة"

information = (
    "الحوسبة الكمومية تعتمد على مبادئ ميكانيكا الكم مثل التراكب والتشابك لحل مشكلات معقدة أسرع بكثير من الحواسيب التقليدية. "
    "شركات مثل IBM وGoogle وD-Wave تحقق تقدماً في بناء معالجات كمومية تتجاوز 100 كيوبِت، بينما أطلقت حكومات عدة "
    "مبادرات وطنية بمليارات الدولارات لدعم هذا المجال. "
    "التطبيقات المتوقعة تشمل تطوير أدوية جديدة عبر محاكاة الجزيئات، تحسين سلاسل الإمداد، تسريع خوارزميات الذكاء الاصطناعي، "
    "وكسر أنظمة التشفير الحالية، ما يثير قضايا أمن معلوماتي. "
    "التحديات الرئيسية تتمثل في تصحيح الأخطاء الكمومية، ضرورة تبريد المعالجات إلى درجات قريبة من الصفر المطلق، "
    "ونقص الكفاءات المتخصصة. "
    "أعلنت IBM عن خارطة طريق لوصول معالجاتها إلى مليون كيوبِت بحلول 2030، بينما تعمل Google على تحقيق “التفوق الكمومي” الثاني "
    "لحل مسائل ذات قيمة عملية حقيقية."
)

host_persona = """ليلى الخالدي - العمر: 45
المهنة/الخلفية: أستاذة جامعية في علوم المستقبل من الدوحة، شغوفة بتبسيط التكنولوجيا للجمهور
الشخصية: فضولية، متفائلة، هادئة النبرة، تحب ربط التكنولوجيا بالتنمية البشرية
OCEAN: انفتاح عالٍ، ضمير عالٍ، انبساط متوسط، وداعة عالية، عصابية منخفضة
أسلوب التحدث: تستخدم أمثلة حياتية، تُدخل اقتباسات علمية، تشرح المصطلحات خطوة بخطوة"""

guest_persona = """حسن المنصوري - العمر: 32
المهنة/الخلفية: رائد أعمال في الحوسبة الكمومية بدبي، أسس شركة ناشئة توفر خدمات Cloud Quantum
الشخصية: حيوي، مجازف، يحب الابتكار، يشرح المفاهيم التقنية بحماس وبساطة
OCEAN: انفتاح عالٍ، ضمير متوسط، انبساط عالٍ، وداعة متوسطة، عصابية منخفضة
أسلوب التحدث: يمزج الفكاهة بالتحليل، يعطي أمثلة من تجربته الريادية، يستخدم مصطلحات كمومية مبسّطة"""

# entertaining style
outline_text = outline_generator.generate_outline(
    topic,
    information,
    host_persona,
    guest_persona,
    style="ترفيهي"
)

print("Generated Outline:\n", outline_text)


Generated Outline:
 {
    "episode_topic": "الحوسبة الكمومية: ثورة التكنولوجيا القادمة",
    "personas": {
        "host": {
            "name": "ليلى الخالدي",
            "background": "أستاذة جامعية في علوم المستقبل من الدوحة، شغوفة بتبسيط التكنولوجيا للجمهور",
            "speaking_style": "تستخدم أمثلة حياتية، تُدخل اقتباسات علمية، تشرح المصطلحات خطوة بخطوة"
        },
        "guest": {
            "name": "حسن المنصوري",
            "background": "رائد أعمال في الحوسبة الكمومية بدبي، أسس شركة ناشئة توفر خدمات Cloud Quantum",
            "speaking_style": "يمزج الفكاهة بالتحليل، يعطي أمثلة من تجربته الريادية، يستخدم مصطلحات كمومية مبسّطة"
        }
    },
    "conversation_flow": {
        "intro1": {
            "description": "مقدمة عامة للبودكاست وترحيب بالمستمعين",
            "tone": "نبرة ترحيبية حماسية",
            "script_elements": [
                "أهلاً وسهلاً بكم في حلقة جديدة من بودكاست التقنية والمستقبل، معكم ليلى الخالدي.",
                "في هذا البودكاست، نتنا

In [15]:
class MSADialogueEnhancer:
    def __init__(self, deployment, model="fanar"):
        self.model = model
        self.deployment = deployment

    def enhance_section(self, section_name, content, criteria, complexity="بسيط", episode_topic="", style_example=None):
        
        complexity_guidance = {
            "بسيط": "استخدم لغة بسيطة ومفهومة للجمهور العام، تجنب المصطلحات المعقدة",
            "متوسط": "امزج بين البساطة والتعمق، استخدم مصطلحات تقنية مع الشرح",
            "معقد": "استخدم مصطلحات متخصصة ومفاهيم متقدمة مع شرح تفصيلي"
        }
        
        # Topic context to avoid confusion
        topic_context = f"""
تذكر أن موضوع الحلقة هو: {episode_topic}
يجب أن يكون كل المحتوى مرتبطاً بهذا الموضوع تحديداً وليس أي موضوع آخر.
"""
        
        if style_example:
            prompt = f"""
هذا مثال على أسلوب الحوار المطلوب:
{style_example}

{topic_context}

بناءً على القسم التالي من البودكاست، قم بتطويره وتحسينه حسب المعايير التالية: {criteria}

مستوى التعقيد المطلوب: {complexity}
{complexity_guidance.get(complexity, '')}

القسم: {section_name}
المحتوى: {content}

المطلوب:
- تأكد أن المحتوى يتعلق بموضوع: {episode_topic}
- اكتب نصاً تفاعلياً وجذاباً باللغة العربية الفصحى
- أضف حشو طبيعي متوسط مثل: يعني، طبعاً، بصراحة، اممم، اههه
- استخدم 70% فصحى و 30% لمسات خليجية خفيفة
- أدرج مراجع ثقافية عند الحاجة
- اجعل النص طبيعياً وتلقائياً
"""
        else:
            prompt = f"""
{topic_context}

بناءً على القسم التالي من البودكاست، قم بتطويره وتحسينه حسب المعايير التالية: {criteria}

مستوى التعقيد المطلوب: {complexity}
{complexity_guidance.get(complexity, '')}

القسم: {section_name}
المحتوى: {content}

المطلوب:
- تأكد أن المحتوى يتعلق بموضوع: {episode_topic} فقط
- اكتب نصاً تفاعلياً وجذاباً باللغة العربية الفصحى
- أضف حشو طبيعي متوسط مثل: يعني، طبعاً، بصراحة، اممم، اههه، واو
- استخدم 70% فصحى و 30% لمسات خليجية خفيفة (مثل: شلون، وش رايك، الله يعطيك العافية)
- أدرج مراجع ثقافية مرتبطة بالموضوع عند الحاجة
- اجعل النص طبيعياً وتلقائياً
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "أنت كاتب سيناريو بودكاست محترف متخصص في المحتوى العربي الطبيعي والتلقائي."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )
        return response.choices[0].message.content

In [16]:
class MSASpontaneousDialogueGenerator:
    def __init__(self, deployment, model="fanar"):
        self.model = model
        self.deployment = deployment
    
    def generate_dialogue(self, section, section_content, host_persona, guest_persona, complexity="بسيط", style="تعليمي", episode_topic="", style_example=None):
        
        complexity_guidance = {
            "بسيط": "اجعل الشرح بسيطاً ومفهوماً للجمهور العام",
            "متوسط": "امزج بين البساطة والعمق، أضف تفاصيل تقنية مع الشرح",
            "معقد": "استخدم مصطلحات متخصصة ونقاش متعمق مع شرح تفصيلي"
        }
        
        style_guidance = {
            "حواري": "ركز على الحوار الطبيعي والتفاعل الشخصي، أضف لحظات تداخل وتفاعل عفوي",
            "تعليمي": "اهتم بتقديم المعلومات بطريقة منظمة ومفصلة، استخدم أمثلة توضيحية",
            "ترفيهي": "أضف عناصر مرحة وقصص شخصية طريفة، استخدم الفكاهة المناسبة",
            "تحليلي": "ركز على التحليل العميق والنقاش المتخصص، استخدم بيانات وإحصائيات"
        }
        
        # MSA fillers for natural conversation
        fillers_guide = """
استخدم حشو المحادثة الطبيعي بكثافة متوسطة:
- تفكير: اممم، اههه، يعني كيف أقول، خلاص
- تأكيد: طبعاً، تماماً، بالضبط، صحيح
- تردد: بعنييييي، يعني، اه ما أدري، شوف
- انفعال: واو، يا الله، ما شاء الله، الله يعطيك العافية
- ربط: بس، لكن، وبعدين، يا أخي، اسمع
- خليجي خفيف: شلون، وش رايك، زين، ماشي الحال
"""
        
        # Topic context to avoid confusion
        topic_context = f"""
تذكر أن موضوع الحلقة هو: {episode_topic}
يجب أن يكون كل الحوار مرتبطاً بهذا الموضوع تحديداً وليس أي موضوع آخر.
"""
        
        if style_example:
            prompt = f"""
هذا مثال على أسلوب الحوار المطلوب:
{style_example}

{topic_context}

حول القسم التالي إلى حوار تلقائي وطبيعي بين المقدم والضيف:

المقدم: {host_persona}
الضيف: {guest_persona}

القسم: {section}
المحتوى: {section_content}

الأسلوب المطلوب: {style}
{style_guidance.get(style, '')}

مستوى التعقيد: {complexity}
{complexity_guidance.get(complexity, '')}

{fillers_guide}

متطلبات الحوار:
- تأكد أن الحوار يدور حول موضوع: {episode_topic}
- استخدم أسماء الشخصيات الفعلية في الحوار
- اجعل كل شخصية تتحدث بأسلوبها المميز
- أضف تفاعلات طبيعية: <happy>, <surprise>, <overlap> (فقط عند الحاجة)
- استخدم 70% فصحى و 30% لمسات خليجية
- أدرج مراجع ثقافية مرتبطة بالموضوع
- اجعل الحوار يتكيف مع الموضوع
- أضف فواصل طبيعية [pause: 2s] عند الحاجة
- ابدأ كل جملة بـ المقدم: أو الضيف:
"""
        else:
            prompt = f"""
{topic_context}

حول القسم التالي إلى حوار تلقائي وطبيعي بين المقدم والضيف باللغة العربية الفصحى:

المقدم: {host_persona}
الضيف: {guest_persona}

القسم: {section}
المحتوى: {section_content}

الأسلوب المطلوب: {style}
{style_guidance.get(style, '')}

مستوى التعقيد: {complexity}
{complexity_guidance.get(complexity, '')}

{fillers_guide}

متطلبات الحوار:
- تأكد أن الحوار يدور حول موضوع: {episode_topic} فقط
- استخدم أسماء الشخصيات الفعلية (أحمد، نور) في الحوار
- اجعل كل شخصية تتحدث بأسلوبها المميز حسب خلفيتها
- أضف تفاعلات طبيعية: <happy>, <surprise>, <overlap> (فقط عند الضرورة)
- استخدم 70% فصحى و 30% لمسات خليجية خفيفة
- أدرج مراجع ثقافية مرتبطة بالموضوع عند الحاجة
- اجعل الحوار يتكيف مع الموضوع
- أضف فواصل طبيعية [pause: 2s] عند الحاجة للتأمل
- ابدأ كل جملة بـ المقدم: أو الضيف:
- اجعل الحوار متماشياً مع الأسلوب المطلوب مع الحفاظ على التلقائية
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "أنت خبير في كتابة الحوارات التلقائية والطبيعية للبودكاست العربي. تخصصك في الأسلوب التعليمي الممتع."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8
        )
        return response.choices[0].message.content

In [17]:
def generate_msa_podcast_script(outline_text, host_persona, guest_persona, complexity="بسيط", style="تعليمي", deployment=None):
    """
    Generate complete MSA podcast script from outline
    
    Args:
        outline_text: JSON string of the podcast outline
        host_persona: Host persona description
        guest_persona: Guest persona description  
        complexity: بسيط/متوسط/معقد (simple/medium/complex)
        style: حواري/تعليمي/ترفيهي/تحليلي (conversational/educational/entertaining/analytical)
        deployment: OpenAI client deployment
    
    Returns:
        List of dictionaries with section dialogues
    """
    
    dialogue_generator = MSASpontaneousDialogueGenerator(deployment, "fanar")
    enhancer = MSADialogueEnhancer(deployment, "fanar")
    
    outline = json.loads(outline_text)
    
    # Extract topic from outline for context
    episode_topic = outline.get("episode_topic", "الموضوع المطروح")
    
    # Complete Arabic dialogue style examples
    arabic_dialogue_styles = {
        "حواري": {
            "host_example": "أحمد: يا أهلاً نور! كيف الحال؟ اممم... قوليلي، شو اللي خلاكِ تدخلي هذا المجال؟ <happy>",
            "guest_example": "نور: أهلاً أحمد! الله يعطيك العافية... يعني بصراحة، هاي قصة طويلة شوي [pause: 2s] بس باختصار، كنت أشوف المشاكل حولي وأقول: ليش ما نحلها بالتقنية؟"
        },
        "تعليمي": {
            "host_example": "أحمد: اممم... نور، ممكن تشرحي لنا بطريقة بسيطة، يعني شلون تشتغل هاي التقنية؟ <happy>",
            "guest_example": "نور: طبعاً أحمد! يعني... اههه كيف أشرح [pause: 2s] تخيل إنك عندك نظام ذكي جداً، بس هذا النظام مش إنسان، هو كمبيوتر! واو صح؟"
        },
        "ترفيهي": {
            "host_example": "أحمد: هاي نور! <happy> قوليلي، إيش أغرب موقف صار معكِ في الشغل؟ يعني شي يضحك؟",
            "guest_example": "نور: ههههه واو أحمد! بصراحة مواقف كثيرة... اممم مرة كنت أجرب البرنامج وفجأة [pause: 2s] خلاص ما عاد يشتغل! قعدت أصرخ: وين راح كودي؟! <surprise>"
        },
        "تحليلي": {
            "host_example": "أحمد: نور، بناءً على الإحصائيات الحديثة، وش رايكِ في التحديات الرئيسية اللي تواجه هذا المجال؟",
            "guest_example": "نور: سؤال ممتاز أحمد... يعني إذا نتكلم بشكل تحليلي، عندنا ثلاث تحديات أساسية [pause: 2s] أولها التقنية، ثانيها التمويل، وثالثها... اممم التقبل المجتمعي"
        }
    }
    
    # Get style examples based on the selected style
    style_examples = arabic_dialogue_styles.get(style, arabic_dialogue_styles['تعليمي'])
    podcast_dialogues = []
    
    # Process conversation flow sections
    conversation_flow = outline.get("conversation_flow", {})
    
    # Intro1
    if "intro1" in conversation_flow:
        intro1_content = conversation_flow["intro1"]
        content_text = " ".join([
            intro1_content.get("opening_line", ""),
            intro1_content.get("podcast_introduction", ""),
            intro1_content.get("episode_hook", "")
        ])
        
        enhanced_intro1 = enhancer.enhance_section(
            "المقدمة الأولى",
            content_text,
            criteria="جذاب، حماسي، ترحيبي، ومتحمس",
            complexity=complexity,
            episode_topic=episode_topic,
            style_example=style_examples["host_example"]
        )
        podcast_dialogues.append({"section": "intro1", "dialogue": enhanced_intro1})
    
    # Intro2  
    if "intro2" in conversation_flow:
        intro2_content = conversation_flow["intro2"]
        content_text = " ".join([
            intro2_content.get("topic_introduction", ""),
            intro2_content.get("guest_welcome", ""),
            intro2_content.get("guest_bio_highlight", "")
        ])
        
        enhanced_intro2 = dialogue_generator.generate_dialogue(
            "تقديم الموضوع والضيف",
            content_text,
            host_persona,
            guest_persona,
            complexity=complexity,
            style=style,
            episode_topic=episode_topic,
            style_example=f"{style_examples['host_example']} {style_examples['guest_example']}"
        )
        podcast_dialogues.append({"section": "intro2", "dialogue": enhanced_intro2})
    
    # Main Discussion Points
    if "main_discussion" in conversation_flow:
        main_points = conversation_flow["main_discussion"]
        
        for i, point in enumerate(main_points):
            point_title = point.get("point_title", f"النقطة {i+1}")
            
            # Build content from all point elements
            content_parts = [
                point.get("personal_angle", ""),
                " ".join(point.get("spontaneous_triggers", [])),
                point.get("disagreement_points", ""),
                " ".join(point.get("cultural_references", [])),
                point.get("natural_transitions", ""),
                point.get("emotional_triggers", "")
            ]
            section_content = " ".join([part for part in content_parts if part])
            
            spontaneous_dialogue = dialogue_generator.generate_dialogue(
                point_title,
                section_content,
                host_persona,
                guest_persona,
                complexity=complexity,
                style=style,
                episode_topic=episode_topic,
                style_example=f"{style_examples['host_example']} {style_examples['guest_example']}"
            )
            
            podcast_dialogues.append({
                "section": f"main_point_{i+1}",
                "title": point_title,
                "dialogue": spontaneous_dialogue
            })
    
    # Closing sections
    closing = conversation_flow.get("closing", {})
    
    # Conclusion
    if "conclusion" in closing:
        conclusion_content = closing["conclusion"]
        content_text = " ".join([
            conclusion_content.get("main_takeaways", ""),
            conclusion_content.get("guest_final_message", ""),
            conclusion_content.get("host_closing_thoughts", "")
        ])
        
        enhanced_conclusion = dialogue_generator.generate_dialogue(
            "الخلاصة",
            content_text,
            host_persona,
            guest_persona,
            complexity=complexity,
            style=style,
            episode_topic=episode_topic,
            style_example=f"{style_examples['host_example']} {style_examples['guest_example']}"
        )
        podcast_dialogues.append({"section": "conclusion", "dialogue": enhanced_conclusion})
    
    # Outro
    if "outro" in closing:
        outro_content = closing["outro"]
        content_text = " ".join([
            outro_content.get("guest_appreciation", ""),
            outro_content.get("audience_thanks", ""),
            outro_content.get("call_to_action", ""),
            outro_content.get("next_episode_teaser", "")
        ])
        
        enhanced_outro = enhancer.enhance_section(
            "الختام",
            content_text,
            criteria="دافئ، ممتن، ملهم، ومشوق للحلقة القادمة",
            complexity=complexity,
            episode_topic=episode_topic,
            style_example=style_examples["host_example"]
        )
        podcast_dialogues.append({"section": "outro", "dialogue": enhanced_outro})
    
    return podcast_dialogues

In [18]:
# Usage example:

# Initialize
dialogue_generator = MSASpontaneousDialogueGenerator(deployment, "fanar")
enhancer = MSADialogueEnhancer(deployment, "fanar")

# Generate script with complexity level
podcast_script = generate_msa_podcast_script(
    outline_text,
    host_persona,
    guest_persona, 
    complexity="متوسط",  # بسيط، متوسط، معقد
    deployment=deployment
)

# Print results
for section in podcast_script:
    print(f"\\n=== {section['section']} ===")
    print(section['dialogue'])


\n=== intro1 ===
### نص البودكاست: المقدمة الأولى

**أحمد:** يا جماعة... اممم... خلونا اليوم نحكي عن موضوع يمكن كثير منا سمع عنه أو شافه في الأخبار بس ما عنده فكرة واضحة عنه. يعني، لو قلت لكم "الحوسبة الكمومية"، شو أول شيء يخطر في بالكم؟ تقنية معقّدة؟ موضوع مستقبلي؟ شيء بعيد عنّا؟ ولا يمكن مجرد اسم كبير بدون معنى واضح؟  

**نور:** أههه، بصراحة أحمد، أنا أتفق معك، يمكن كثير ناس يسمعون الكلمة ويقولون: "واو، هذا شيء للعلماء الفيزياء والرياضيات، مالنا دخل فيه." بس الواقع؟ الحوسبة الكمومية مو مجرد أجهزة معقدة في مختبرات مغلقة، هي فعلاً ثورة قادمة، شيء بيغيّر تماماً الطريقة اللي نفكر فيها، نحسب فيها، وحتى، يمكن، الطريقة اللي نعيش فيها حياتنا اليومية.  

**أحمد:** يعني، لما تقولين "ثورة"، تقصدين إيه بالضبط؟ لأننا بصراحة تعودنا نقول عن أي تقنية جديدة إنها "ثورة"، مثل لما طلعت الهواتف الذكية أو لما الإنترنت صار جزء من حياتنا. فالحوسبة الكمومية... ليش نعتبرها مختلفة؟  

**نور:** أههه، سؤال ممتاز، أحمد. تخيل معي، يعني، لو عندك جهاز كمبيوتر عادي، مثل اللي تستخدمه يومياً، هذا الجهاز يعتمد على شيء ب